## 1. Descargar la base de datos proporcionada en el formato de preferencia
## 2. Importar las librerías necesarias:

In [1]:

import pandas as pd
import numpy as np
import plotly.express as px

df = pd.read_excel('./data/data.xlsx', sheet_name='Base dsp-analytics-daily')
print(df.shape)
df.head()

(4004, 12)


,RETAIL,CATEGORIA,Date,Impressions,Clicks,CTR,Unique Visitors,Frequency,Spend,Units,Sales,ROAS
0,BA,Cremas,2023-07-01,2386,14,0.0059,2391.0,0.9979,134.40,13,405.0,3.01
1,BA,Cremas,2023-07-02,3423,17,0.0050,3319.0,1.0313,163.20,17,561.5,3.44
2,BA,Cremas,2023-07-03,4241,17,0.0040,4242.0,0.9998,136.32,13,427.5,3.14
3,BA,Cremas,2023-07-04,3947,20,0.0051,3913.0,1.0087,153.60,8,272.0,1.77
4,BA,Cremas,2023-07-05,3697,19,0.0051,3707.0,0.9973,105.72,21,3903.0,36.92


#### Se dividen las variables cunatitativas de las cualitativas

In [2]:
cualitative = ['RETAIL', 'CATEGORIA']
cuantitive = [i for i in df.columns if i not in cualitative and i != "Date"]
print(cuantitive)

['Impressions', 'Clicks', 'CTR', 'Unique Visitors', 'Frequency', 'Spend', 'Units', 'Sales', 'ROAS']


## 3. Limpiar y transformar los datos:


In [3]:
df.describe().T.style.background_gradient(cmap='coolwarm')

,count,mean,min,25%,50%,75%,max,std
Date,4004,2023-05-03 17:52:05.274725120,2023-01-01 00:00:00,2023-03-11 00:00:00,2023-05-10 00:00:00,2023-06-28 00:00:00,2023-08-11 00:00:00,nan
Impressions,4004.000000,3181.110390,0.000000,0.000000,1049.000000,3820.500000,133758.000000,6496.049136
Clicks,4004.000000,64.187812,0.000000,0.000000,25.000000,75.000000,2083.000000,120.062119
CTR,4004.000000,0.022348,0.000000,0.000000,0.015700,0.030700,0.500000,0.028466
Unique Visitors,2843.000000,2073.557861,1.000000,441.500000,1158.000000,2348.500000,126706.000000,5207.270286
Frequency,2843.000000,2.232768,0.000000,1.496550,1.982800,2.820500,8.418800,0.956514
Spend,4004.000000,643.133229,0.000000,0.000000,274.000000,802.850000,12721.000000,1068.795755
Units,4004.000000,97.476024,-1.000000,9.000000,28.000000,79.250000,3701.000000,240.470749
Sales,4004.000000,8878.259888,-262.990000,560.645000,1739.500000,5787.375000,854804.910000,30143.257392
ROAS,4004.000000,7.549788,0.000000,0.000000,3.420000,7.162500,594.400000,21.235721


#### Se recalcula el CTR con la siguiente formula 
$$
\text{CTR} = \frac{\text{Clics}}{\text{Impresiones}}
$$

In [4]:

df["CTR"] = df.apply(lambda x: x["Clicks"] / x["Impressions"] if x["Impressions"] != 0 else 0, axis=1)
print(df["Impressions"].isna().sum())
print(df["Clicks"].isna().sum())
print(df["CTR"].isna().sum())
df["CTR"].describe()


0
0
0


count    4004.000000
mean        0.022348
std         0.028466
min         0.000000
25%         0.000000
50%         0.015740
75%         0.030655
max         0.500000
Name: CTR, dtype: float64

In [5]:

px.histogram(df, x = "CTR")

#### Se infiere que no debe haber unidades negativas, por lo que se le aplica el valor absoluto

In [6]:
df["Units"] = df["Units"].apply(lambda x: np.absolute(x))
df["Units"].describe().T

count    4004.000000
mean       97.476523
std       240.470546
min         0.000000
25%         9.000000
50%        28.000000
75%        79.250000
max      3701.000000
Name: Units, dtype: float64

#### Se modifica el tipo de dato de la columna "Date"

In [7]:
df["Date"] = df["Date"].astype("datetime64[ns]")

#### Se estandarizan las categorias

In [8]:
df["CATEGORIA"].unique()

array(['Cremas', 'Leches', 'Postres', 'Quesos', 'Yogurt',
       'AlimentoLiquido', 'AlimentoLIquido', 'Carnicos', 'Crema', 'H&W',
       'Jugos', 'Leche', 'Queso', 'Reyes', 'Formulas', 'Yoghurt',
       'Yogurth', 'Fórmulas', 'dic', 'Vita', 'AlimLiq', 'Cuaresma',
       'Form', 'Lanzamientos', 'Mantequilla', 'SanValentin', 'SuperBowl',
       'YogurtGriego', 'Aliquido'], dtype=object)

In [9]:
def eliminar_acentos(text):
    a,b = 'áéíóúü','aeiouu'
    aux = str.maketrans(a,b)
    return text.translate(aux)

def correct_category (text):
    aux = text.lower()
    aux = eliminar_acentos(aux)
    if "yog" in aux:
        return "yogurt"
    elif "liq" in aux:
        return "alimento liquido"
    elif "form" in aux:
        return "formula"
    elif "crem" in aux:
        return "crema"
    elif "queso" in aux:
        return "queso"
    elif "leche" in aux:
        return "leche"
    else:
        return aux
df["CATEGORIA"] = df["CATEGORIA"].apply(lambda x: correct_category(x))

In [10]:
df = df[df['CATEGORIA'] != 'dic']


#### Para este caso se dejaran los valores atipicos observados

## 4. Eliminar filas duplicadas.

#### No se encuentran valores duplicados al no existir algun identificador que pudirean sugerirlo

In [11]:
print(df.shape)
df.drop_duplicates()
df.shape

(3976, 12)


(3976, 12)

## 5. Corregir valores inconsistentes.


#### Se realizó en el paso 3

## 6. Tratar valores nulos.


In [12]:
df.isna().sum()


RETAIL                0
CATEGORIA             0
Date                  0
Impressions           0
Clicks                0
CTR                   0
Unique Visitors    1134
Frequency          1134
Spend                 0
Units                 0
Sales                 0
ROAS                  0
dtype: int64

#### Podriamos borrarlos simplemente

In [13]:
df_without_na = df.dropna()
df_without_na.isna().sum()

RETAIL             0
CATEGORIA          0
Date               0
Impressions        0
Clicks             0
CTR                0
Unique Visitors    0
Frequency          0
Spend              0
Units              0
Sales              0
ROAS               0
dtype: int64

#### O podriamos llenarlos con alguna metrica como la media

In [14]:
mean_v = df_without_na["Unique Visitors"].mean()
mean_f = df_without_na["Frequency"].mean()

print (f""" Unique Visitors Mean: {mean_v}
Frequency Mean: {mean_f}
""")

df_fill_with_mean = df.fillna({"Unique Visitors": mean_v, "Frequency": mean_f})
df_fill_with_mean.isna().sum()

 Unique Visitors Mean: 2074.2860661505983
Frequency Mean: 2.233201794510908



RETAIL             0
CATEGORIA          0
Date               0
Impressions        0
Clicks             0
CTR                0
Unique Visitors    0
Frequency          0
Spend              0
Units              0
Sales              0
ROAS               0
dtype: int64

In [15]:
df = df_without_na

# 7. Cree dos columnas adicionales, dia y mes

In [16]:

df["Month"] = df["Date"].dt.month
df["Day"] = df["Date"].dt.day
df.head()

,RETAIL,CATEGORIA,Date,Impressions,Clicks,CTR,Unique Visitors,Frequency,Spend,Units,Sales,ROAS,Month,Day
0,BA,crema,2023-07-01,2386,14,0.005868,2391.0,0.9979,134.40,13,405.0,3.01,7,1
1,BA,crema,2023-07-02,3423,17,0.004966,3319.0,1.0313,163.20,17,561.5,3.44,7,2
2,BA,crema,2023-07-03,4241,17,0.004008,4242.0,0.9998,136.32,13,427.5,3.14,7,3
3,BA,crema,2023-07-04,3947,20,0.005067,3913.0,1.0087,153.60,8,272.0,1.77,7,4
4,BA,crema,2023-07-05,3697,19,0.005139,3707.0,0.9973,105.72,21,3903.0,36.92,7,5


## 8. Cree una columna con convertir las impresiones en miles (000)



In [17]:
df["New Impressions"] = df["Impressions"].apply(lambda x: x*1000)
df.head()

,RETAIL,CATEGORIA,Date,Impressions,Clicks,CTR,Unique Visitors,Frequency,Spend,Units,Sales,ROAS,Month,Day,New Impressions
0,BA,crema,2023-07-01,2386,14,0.005868,2391.0,0.9979,134.40,13,405.0,3.01,7,1,2386000
1,BA,crema,2023-07-02,3423,17,0.004966,3319.0,1.0313,163.20,17,561.5,3.44,7,2,3423000
2,BA,crema,2023-07-03,4241,17,0.004008,4242.0,0.9998,136.32,13,427.5,3.14,7,3,4241000
3,BA,crema,2023-07-04,3947,20,0.005067,3913.0,1.0087,153.60,8,272.0,1.77,7,4,3947000
4,BA,crema,2023-07-05,3697,19,0.005139,3707.0,0.9973,105.72,21,3903.0,36.92,7,5,3697000


## 9. Agregar una columna y calcular el Costo por Unidad

In [18]:
df["Unit Cost"] = df.apply(lambda x: x["Sales"]/x["Units"] if x["Units"] > 0 else 0, axis=1)
df["Unit Cost"].isna().sum()

np.int64(0)

## 10. Cree una columna llamada "Best Practice", y si el ROAS (Retorno sobre la Inversión) es mayor a 20, identificar los registros en esta columna como "BP"


In [19]:
df["Best Practice"] = df["ROAS"].apply(lambda x: "BP" if x >= 20.0 else "No BP") 
df.head()

,RETAIL,CATEGORIA,Date,Impressions,Clicks,CTR,Unique Visitors,Frequency,Spend,Units,Sales,ROAS,Month,Day,New Impressions,Unit Cost,Best Practice
0,BA,crema,2023-07-01,2386,14,0.005868,2391.0,0.9979,134.40,13,405.0,3.01,7,1,2386000,31.153846,No BP
1,BA,crema,2023-07-02,3423,17,0.004966,3319.0,1.0313,163.20,17,561.5,3.44,7,2,3423000,33.029412,No BP
2,BA,crema,2023-07-03,4241,17,0.004008,4242.0,0.9998,136.32,13,427.5,3.14,7,3,4241000,32.884615,No BP
3,BA,crema,2023-07-04,3947,20,0.005067,3913.0,1.0087,153.60,8,272.0,1.77,7,4,3947000,34.000000,No BP
4,BA,crema,2023-07-05,3697,19,0.005139,3707.0,0.9973,105.72,21,3903.0,36.92,7,5,3697000,185.857143,BP


## 11. Cree una tabla de datos organizando de mayor a menor, las categorias de esta base.


In [20]:
df_sorted = df.sort_values(by="CATEGORIA",ascending=False)
df_sorted.head()

,RETAIL,CATEGORIA,Date,Impressions,Clicks,CTR,Unique Visitors,Frequency,Spend,Units,Sales,ROAS,Month,Day,New Impressions,Unit Cost,Best Practice
4003,WM,yogurt,2023-08-11,4139,115,0.027784,962.0,4.3025,806.45,155,4396.00,5.45,8,11,4139000,28.361290,No BP
1645,SAMS,yogurt,2023-01-18,712,35,0.049157,334.0,2.1317,476.19,3,258.93,0.54,1,18,712000,86.310000,No BP
942,SAMS,yogurt,2023-02-03,3376,76,0.022512,1556.0,2.1697,1000.00,14,1423.00,1.42,2,3,3376000,101.642857,No BP
943,SAMS,yogurt,2023-02-04,3751,74,0.019728,1815.0,2.0667,1000.00,26,2616.10,2.62,2,4,3751000,100.619231,No BP
944,SAMS,yogurt,2023-02-05,3245,74,0.022804,1579.0,2.0551,1000.00,27,2698.70,2.70,2,5,3245000,99.951852,No BP


## 12. Cree una tabla con % del gasto total que se invirtió para cada categoría 


In [35]:

total = df["Spend"].sum()
print(total)
df_grouped = df.groupby("CATEGORIA")["Spend"].sum()
df_grouped = df_grouped.apply(lambda x: (x/total)*100)
df_grouped


2575105.45


CATEGORIA
alimento liquido     2.241844
carnicos             3.546619
crema               11.260547
cuaresma             2.169204
formula              3.053503
h&w                  0.086297
jugos                0.078568
lanzamientos         1.153901
leche               42.006531
mantequilla          1.299744
postres              2.899423
queso               11.349819
reyes                3.373018
sanvalentin          0.183022
superbowl            0.737835
vita                 3.013098
yogurt              11.547028
Name: Spend, dtype: float64

In [36]:
px.pie(df_grouped, values="Spend", names=df_grouped.index, title="Porcentaje de gasto por categoría")

## 13. Guardar la base de datos transformada.
 

In [37]:
df.to_csv('./data/output.csv', index=False)